In [1]:
import pandas as pd
from commcare_export.checkpoint import CheckpointManagerWithDetails
from commcare_export.commcare_hq_client import CommCareHqClient, AUTH_MODE_APIKEY
from commcare_export.commcare_minilinq import get_paginator, PaginationMode
from datetime import datetime
import re

# CommCare credentials
username = 'amutenha@zimttech.org'  # Replace with your actual CommCare username
domain = 'zdip-qa'  # Your CommCare project domain
hq_host = 'https://zdip.zimttech.org'  # Your CommCare HQ custom host
API_KEY = '548bb49381030201a991d1e801a2a6fb1332b5f5'  # Replace with the API key generated in CommCare


zazicSites= pd.read_excel('zazicFacilities.xlsx')
# Initialize API client
api_client = CommCareHqClient(hq_host, domain, username, API_KEY, AUTH_MODE_APIKEY)

# Create a paginator for the 'form' resource (fetching data from a form)
form_paginator = get_paginator(resource='form', pagination_mode=PaginationMode.date_modified, page_size=100)
form_paginator.init()

# Initialize checkpoint manager (None used here for simplicity)
checkpoint_manager = CheckpointManagerWithDetails(None, None, PaginationMode.date_modified)

# Date range for filtering
start_date = datetime(2024, 12,6)  # 2 December 2024
end_date = datetime(2024, 12, 25)  # 6 December 2024

# Specify the app_id to filter forms
desired_app_id ='cbd88c81d5a9131517e106028f0dd680'  

# Initialize lists to store categorized data
referrals_data = []
AEs_data = []
general_data = []

# Fetch forms iteratively
# Fetch forms using manual pagination
all_forms = []
offset = 0
while True:
    forms = api_client.iterate(
        'form',
        form_paginator,
        checkpoint_manager=checkpoint_manager,
        params={'limit': 20, 'offset': offset}  # Adjust offset manually
    )
    
    batch = list(forms)  # Fetch current batch of forms
    if not batch:  # Break if no more forms
        break
    
    all_forms.extend(batch)
    offset += 20  # Move to the next page

    # Optional: Stop after collecting 100 submissions for testing
    if len(all_forms) >= 100:
        break

# print(f"Fetched {len(all_forms)} submissions.")

filteredResults =[]
for i in all_forms:
    if i["app_id"] ==desired_app_id:
        filteredResults.append(i)

print(filteredResults)
# Filter and categorize forms by app_id and date
for form in forms:
    if form.get('app_id') == desired_app_id:
        print(form)
        form_received_date = datetime.strptime(form['received_on'], "%Y-%m-%dT%H:%M:%S.%fZ")
        if start_date <= form_received_date <= end_date:
            general_data.append(form)
# Convert each list to a pandas DataFrame
general_df = pd.json_normalize(general_data)
general_df.to_excel('test.xlsx')

general_df.columns = [re.split(r'[./]', col)[-1] for col in general_df.columns]

# New columns that should NOT be deleted
columns_to_retain = [
    'AERecordingMonth', 'AERecordingSite', 'AERecordingYear', 'aeComments', 'ae_classification', 
    'ae_type_code', 'checkMonthValidation', 'checkYearValidation', 'circumcising_cadre', 'client_age', 
    'date_ae_identified', 'mcMethod', 'vmmc_number', 'followUpTotal', 'recordingMonth', 
    'total_hiv_negative_linked_to_prep', 'total_hiv_positive_linked_to_care', 'ReferrelRecordingMonth', 
    'ReferrelRecordingSite', 'ReferrelRecordingYear', 'reason_for_referral', 'services_referred', 'facilityName', 
    'District', 'recordingMonth', 'year', 'totalAES','adverse_events', 'Site_Name'
]

# Columns that must be numeric (including the new ones)
numeric_columns = [
    'client_age', 'fu15-19', 'fu20-24', 'fu25-29', 'fu30-34', 'fu40-44', 'fu50',
    'total_hiv_negative_linked_to_prep', 'total_hiv_positive_linked_to_care', 'referralToOtherServices',
    'total_mcs_referred_for_srh_services', 'total_mcs_referred_for_sti_services', 'checkifFacilityHasBeenSaved',
    'hivNegativeNC15-19', 'hivNegativeNC20-24', 'hivNegativeNC25-29', 'hivNegativeNC30-34', 'hivNegativeNC35-39', 
    'hivNegativeNC40-44', 'hivNegativeNC45-49', 'hivNegativeNC50', 'total_surgicalDisposable', 'hivPositive30-34',
    'hivPositiveNC15-19', 'hivPositiveNC20-24', 'hivPositiveNC25-29', 'hivPositiveNC35-39', 'hivPositiveNC40-44',
    'hivPositiveNC45-49', 'hivPositiveNC50', 'total15-19NCHIVTested', 'total20-24NCHIVTested', 'total25-29NCHIVTested',
    'total30-34NCHIVTested', 'total35-39NCHIVTested', 'total40-44NCHIVTested', 'total45-49NCHIVTested', 
    'total50NCHIVTested', 'total_surgicalReusable', 'totalHIVPositiveNC', 'totalhivNegativeNC', 
    'uncircumcisedClientsForHTS', 'hivNegative15-19', 'hivNegative20-24', 'hivNegative25-29', 'hivNegative30-34', 
    'hivNegative40-44', 'hivNegative50', 'total_surgicalDisposable', 'hivPositive15-19', 'hivPositive20-24','hivPositive25-29',
    'hivPositive30-34','hivPositive35-39','hivPositive40-44','hivPositive45-49','hivPositive50',
    'total15-19HIVTested', 'total20-24HIVTested', 'total25-29HIVTested', 'total30-34HIVTested', 'total35-39HIVTested',
    'total40-44HIVTested', 'total45-49HIVTested', 'total50HIVTested', 'total_surgicalReusable', 'totalHIVPositive',
    'totalHIVUntested', 'totalhivNegative', 'TotalMCsBYMethod', 'sgDisposable15-19', 'sgDisposable20-24', 
    'sgDisposable25-29', 'sgDisposable30-34', 'sgDisposable40-44', 'sgDisposable50', 'total_surgicalDisposable',
    'sgReusable15-19', 'sgReusable20-24', 'sgReusable25-29', 'sgReusable30-34', 'sgReusable40-44', 'sgReusable50',
    'total_surgicalReusable', 'total50ByMethod', 'totalShangringMCs', 'totalSurgicalDisposableMCs', 
    'totalSurgicalReusableMCs', 'mc15-19', 'mc20-24', 'mc25-29', 'mc30-34', 'mc35-39', 'mc40-44', 'mc45-49', 
    'mc50', 'totalMCs', 'fu35-39', 'other_referrals_in_detail', 'total_mcs_referred_to_other_services', 
    'hivNegative35-39', 'sgDisposable35-39', 'sgReusable35-39', 'shangring15-19', 'shangring20-24','shangring25-29', 'shangring30-34',
    'shangring35-39', 'shangring40-44','shangring45-49','shangring50', 'fu45-49', 
    'hivNegative45-49', 'sgDisposable45-49', 'sgReusable45-49','AERecordingYear'
]

# general_df['AERecordingYear'] = pd.to_numeric(general_df['AERecordingYear'])
# Combine the two lists (retain columns that are in either columns_to_retain or numeric_columns)
columns_to_keep = list(set(columns_to_retain + numeric_columns))

# Identify missing columns
missing_columns = [col for col in columns_to_keep if col not in general_df.columns]

# Add missing columns to the DataFrame and assign zero
for col in missing_columns:
    general_df[col] = 0

# Drop the columns that are not in the columns_to_keep list
# Sort the columns alphabetically in both DataFrames

# For the 'general_df', sort the columns alphabetically
general_df_sorted = general_df[sorted(general_df[columns_to_keep])]

# For the AE-specific columns, we can do the same
ae_columns = [
    'aeComments', 'ae_type_code', 'date_ae_identified', 'AERecordingYear', 'AERecordingSite', 
    'client_age', 'circumcising_cadre', 'ae_classification', 'AERecordingMonth', 'mcMethod','vmmc_number'
]


referrals_columns =['ReferrelRecordingMonth', 
    'ReferrelRecordingSite', 'ReferrelRecordingYear', 'services_referred','reason_for_referral']



other_referrals = []
ae_data_df_sorted = general_df[ae_columns][sorted(ae_columns)]
ae_data_df_sorted = ae_data_df_sorted[ae_data_df_sorted['AERecordingMonth'].notna()]

referralsDF_sorted = general_df[referrals_columns][sorted(referrals_columns)]
referralsDF_sorted = referralsDF_sorted[referralsDF_sorted['ReferrelRecordingMonth'].notna()]

# Function to flatten the nested referral data
def flatten_referral_data(referral_list):
    flattened_referrals = []
    
    # Loop through each referral in the list
    for referral in referral_list:
        flattened_referral = {}
        
        # Flatten the first level of referral data
        for key, value in referral.items():
            if isinstance(value, dict):
                # If the value is a nested dictionary, flatten it
                for nested_key, nested_value in value.items():
                    flattened_referral[f"{key}_{nested_key}"] = nested_value
            else:
                flattened_referral[key] = value
        
        flattened_referrals.append(flattened_referral)
    
    return flattened_referrals



# Function to unpack nested 'question1_question5' field
def unpack_detailedField(df, column):
    if column  in df.columns:
        # Unpack the dictionary inside 'question1_question5' into separate columns
        question_fields = df[column].apply(pd.Series)
        
        # Merge the unpacked fields back into the main DataFrame
        df = pd.concat([df, question_fields], axis=1)
        
        # Drop the original 'question1_question5' column if no longer needed
        df.drop(column, axis=1, inplace=True)
    
    return df



# Extract the referrals from 'other_referrals_in_detail' column referrals_data = []
singleReferrals =[]
# Loop through each row and check if 'other_referrals_in_detail' is a valid list
for idx, row in general_df.iterrows():
    referral_list = row['other_referrals_in_detail']
    
    # Check if the value is a list and not NaN
    if isinstance(referral_list, list):
        # Flatten the referral data
        flattened_referrals = flatten_referral_data(referral_list)
        for ref in flattened_referrals:
            ref['District'] = row['District']  # Add the District to each referral record
            referrals_data.append(ref)
    else:
        # Handle cases where referral_list is not valid
        if not pd.isna(row['ReferrelRecordingMonth']):  # Check if 'ReferrelRecordingMonth' is not NaN
            _object = {
                'ReferrelRecordingMonth': row['ReferrelRecordingMonth'], 
                'ReferrelRecordingSite': row['ReferrelRecordingSite'], 
                'ReferrelRecordingYear': row['ReferrelRecordingYear'],
                'District': row['District'],
                'reason_for_referral':row['reason_for_referral'],
                'services_referred':row['services_referred']
            }
            singleReferrals.append(_object)






aes_data =[]
singleAEs = []
for idx, row in general_df.iterrows():
    ae_list = row['adverse_events']
    
    # Check if the value is a list and not NaN
    if isinstance(ae_list, list) and not pd.isna(ae_list).any():  # Use .any() to check the entire list
        flattened_aes = flatten_referral_data(ae_list)
        for ref in flattened_aes:
            ref['District'] = row['District']  # Add the District to each referral record
            aes_data.append(ref)

    else:
        if not pd.isna(row['AERecordingMonth']):
            _object = {
                'AERecordingMonth':row['AERecordingMonth'],
                'AERecordingSite':row['AERecordingSite'],
                'AERecordingYear':row['AERecordingYear'],
                'District':row['District'],
                'aeComments':row['aeComments'],
                'ae_classification':row['ae_classification'],
                'ae_type_code':row['ae_type_code'],
                'circumcising_cadre':row['circumcising_cadre'],
                'client_age':row['client_age'],
                'date_ae_identified':row['date_ae_identified'],
                'mcMethod':row['mcMethod'],
                'vmmc_number':row['vmmc_number']
            }
            singleAEs.append(_object)




            
            
# Convert the flattened referrals data into a DataFrame

referrals_df = pd.DataFrame(referrals_data)

# convert the flattened aes data into a df

ae_df = pd.DataFrame(aes_data)

# Unpack the 'question1_question5' field
referrals_df = unpack_detailedField(referrals_df,'question1_question5')

ae_df = unpack_detailedField(ae_df,'question3_question1')

ae_df = unpack_detailedField(ae_df,'question1')

ae_df = ae_df.rename(columns={
    'question3_AERecordingMonth': 'AERecordingMonth',
    'question3_AERecordingSite': 'AERecordingSite',
    'question3_AERecordingYear': 'AERecordingYear'
})

print(ae_df)

if len(singleReferrals)>0:
    tempRefDF = pd.DataFrame(singleReferrals)
    referrals_df = pd.concat([referrals_df,tempRefDF])

if len(singleAEs)>0:
    tempRefDF = pd.DataFrame(singleAEs)
    ae_df = pd.concat([ae_df,tempRefDF])


if not referrals_df.empty:
    referrals_df = referrals_df.drop(columns = ['question1_cancelthisReferralReport',0])

if not ae_df.empty:
    ae_df = ae_df.drop(columns = ['checkMonthValidation','checkYearValidation'])

general_df_sorted = general_df_sorted.drop(columns =['adverse_events','other_referrals_in_detail'])


zazicSites_unique = zazicSites.drop_duplicates(subset='facilityName')

# Create the mapping for unique values
facility_mapping = zazicSites_unique.set_index('facilityName')['facilityType']

# Map facilityType to general_df_sorted
general_df_sorted['facilityType'] = general_df_sorted['facilityName'].map(facility_mapping)

                
# Now save the DataFrame to Excel
with pd.ExcelWriter('data.xlsx', engine='xlsxwriter') as writer:
    # Save the sorted general data to the first sheet
    general_df_sorted.to_excel(writer, sheet_name='Statistics', index=False)
    
    # Save the sorted AE data to a new sheet
    ae_df.to_excel(writer, sheet_name='AEs', index=False)
    
    # Save the flattened referrals data to a new sheet
    referrals_df.to_excel(writer, sheet_name='Referrals', index=False)

[{'app_id': 'cbd88c81d5a9131517e106028f0dd680', 'archived': False, 'attachments': {'form.xml': {'content_type': 'text/xml', 'length': 5750, 'url': 'https://zdip.zimttech.org/a/zdip-qa/api/form_attachment/v1/3b25ab17-06a1-417b-a61a-7cca40db62dc/form.xml'}}, 'build_id': '18b954b4cbe646a2bbc8a06b0c420f6e', 'domain': 'zdip-qa', 'edited_by_user_id': None, 'edited_on': None, 'form': {'#type': 'data', '@name': 'ZAZIC  Data Collection Form', '@uiVersion': '1', '@version': '395', '@xmlns': 'http://openrosa.org/formdesigner/2C62E5DB-A039-4F7E-A378-25101817B038', 'client_follow-upGRP': {'follow-up_by_age_disaggregation': {'copy-1-of-mcs_by_age_disaggregations': 'OK', 'followUpTotal': '', 'fu15-19': '25', 'fu20-24': '15', 'fu25-29': '19', 'fu30-34': '14'}}, 'commcare_usercase': {'case': {'@case_id': 'a80c46d3231640fd8c55f749973adc25', '@date_modified': '2024-12-13T07:21:26.649000Z', '@user_id': '41c47d57bc31473c8e8c8d8cf3a78062', '@xmlns': 'http://commcarehq.org/case/transaction/v2', 'update': {'f

{'app_id': 'cbd88c81d5a9131517e106028f0dd680', 'archived': False, 'attachments': {'form.xml': {'content_type': 'text/xml', 'length': 5750, 'url': 'https://zdip.zimttech.org/a/zdip-qa/api/form_attachment/v1/3b25ab17-06a1-417b-a61a-7cca40db62dc/form.xml'}}, 'build_id': '18b954b4cbe646a2bbc8a06b0c420f6e', 'domain': 'zdip-qa', 'edited_by_user_id': None, 'edited_on': None, 'form': {'#type': 'data', '@name': 'ZAZIC  Data Collection Form', '@uiVersion': '1', '@version': '395', '@xmlns': 'http://openrosa.org/formdesigner/2C62E5DB-A039-4F7E-A378-25101817B038', 'client_follow-upGRP': {'follow-up_by_age_disaggregation': {'copy-1-of-mcs_by_age_disaggregations': 'OK', 'followUpTotal': '', 'fu15-19': '25', 'fu20-24': '15', 'fu25-29': '19', 'fu30-34': '14'}}, 'commcare_usercase': {'case': {'@case_id': 'a80c46d3231640fd8c55f749973adc25', '@date_modified': '2024-12-13T07:21:26.649000Z', '@user_id': '41c47d57bc31473c8e8c8d8cf3a78062', '@xmlns': 'http://commcarehq.org/case/transaction/v2', 'update': {'fa

  AERecordingMonth AERecordingSite AERecordingYear     District  \
0         November    Krima Clinic            2024  GOKWE SOUTH   
1         November    Krima Clinic            2024  GOKWE SOUTH   

  cancelthisAEReport      aeComments ae_classification ae_type_code  \
0                     Client healing            Severe          COA   
1                     Client healing          Moderate           BL   

  checkMonthValidation checkYearValidation circumcising_cadre client_age  \
0                    1                   1              Nurse         25   
1                    1                   1              Nurse         18   

  date_ae_identified  mcMethod vmmc_number  
0         2024-11-10  Surgical      467233  
1         2024-11-08  Surgical      245555  


In [2]:
general_df_sorted

,AERecordingMonth,AERecordingSite,AERecordingYear,District,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,Site_Name,TotalMCsBYMethod,aeComments,...,total_surgicalReusable,total_surgicalReusable,total_surgicalReusable,total_surgicalReusable,totalhivNegative,totalhivNegativeNC,uncircumcisedClientsForHTS,vmmc_number,year,facilityType
0,NaN,NaN,NaN,GOKWE SOUTH,December,Gokwe South District Hospital,2024,GOKWE - 100513 - DISTRICT HOSPITAL,73,NaN,...,NaN,OK,OK,NaN,69,0,0,NaN,2024,Static
1,December,Gwanyika Rural Health Centre,2024,GOKWE SOUTH,December,Gwanyika Rural Health Centre,2024,GOKWE - 100513 - DISTRICT HOSPITAL,37,Client is healing,...,NaN,OK,OK,NaN,37,0,0,345534,2024,Outreach
2,NaN,NaN,NaN,GOKWE SOUTH,NaN,NaN,NaN,GOKWE - 100513 - DISTRICT HOSPITAL,25,NaN,...,NaN,OK,NaN,NaN,25,0,0,NaN,2024,Outreach
3,November,Katema RDC,2024,GOKWE SOUTH,NaN,NaN,NaN,GOKWE - 100513 - DISTRICT HOSPITAL,94,Client is healing,...,NaN,OK,OK,NaN,13,0,0,234556,2024,Outreach
4,November,Jahana Clinic,2024,GOKWE SOUTH,NaN,NaN,NaN,GOKWE - 100513 - DISTRICT HOSPITAL,80,"Sature elavated by lignocaine,\nclient healing",...,OK,OK,OK,OK,80,1,1,234455,2024,Outreach
5,NaN,NaN,NaN,GOKWE SOUTH,NaN,NaN,NaN,GOKWE - 100513 - DISTRICT HOSPITAL,35,NaN,...,NaN,OK,OK,NaN,35,0,0,NaN,2024,Outreach
6,November,Huchu Clinic,2024,GOKWE SOUTH,November,Huchu Clinic,2024,GOKWE - 100513 - DISTRICT HOSPITAL,60,Client is healing,...,OK,OK,OK,OK,59,0,1,00056,2024,Outreach
